In [2]:
import numpy as np
import pandas as pd
import math as m
import cmath
import sympy as sym
import scipy as sci
import matplotlib.ticker as ticker
import random as rand
import matplotlib.pyplot as plt
import cmath as cm
from IPython.display import display, Latex
from sympy import pprint
from scipy.optimize import minimize as GDlib
import MLE_functions as fn

In [3]:
theta1, theta2, phi1, phi2, alpha = sym.symbols('theta1 theta2 phi1 phi2 alpha')
c1, c2, s1, s2, w1, w2 = sym.cos(theta1), sym.cos(theta2), sym.sin(theta1), sym.sin(theta2), sym.cos(alpha)**2, sym.sin(alpha)**2

# # analytic formuale for the probabilities, as given in the notes
pr1 = (1/6)*(w1*s1**2*(1+c1**2 -2*m.sqrt(2)*sym.cos(phi1)*s1*c1)+w2*s2**2*(1+c2**2-2*m.sqrt(2)*sym.cos(phi2)*s2*c2))
pr2 = (1/6)*(w1*c1**2*(1+s1**2 -2*m.sqrt(2)*sym.cos(phi1)*c1*s1) + w2*c2**2*(1+s2**2*-2*m.sqrt(2)*sym.cos(phi2)*c2*s2))
pr3 = (1/6)*(w1*(c1**4+s1**4 -2*sym.cos(2*phi1)*s1**2*c1**2)+ w2*(c2**4+ s2**4 - 2*sym.cos(2*phi2)*s2**2*c2**2))
pr4 = (1/6)*(w1*s1**2*(1+c1**2 -2*m.sqrt(2)*sym.cos(phi1+2*m.pi/3)*s1*c1)+ w2*s2**2*(1+c2**2-2*m.sqrt(2)*sym.cos(phi2+2*m.pi/3)*s2*c2))
pr5= (1/6)*(w1*c1**2*(1+s1**2 -2*m.sqrt(2)*sym.cos(phi1-2*m.pi/3)*c1*s1)+ w2*c2**2*(1+s2**2-2*m.sqrt(2)*sym.cos(phi2-2*m.pi/3)*c2*s2))
pr6 = (1/6)*(w1*(c1**4+s1**4-2*sym.cos(2*phi1 - 2*m.pi/3)*c1**2*s1**2)+ w2*(c2**4+s2**4-2*sym.cos(2*phi2 - 2*m.pi/3)*c2**2*s2**2))
pr7 = (1/6)*(w1*s1**2*(1+c1**2-2*m.sqrt(2)*sym.cos(phi1-2*m.pi/3)*s1*c1)+ w2*s2**2*(1+c2**2-2*m.sqrt(2)*sym.cos(phi2-2*m.pi/3)*s2*c2))
pr8 = (1/6)*(w1*c1**2*(1+s1**2-2*m.sqrt(2)*sym.cos(phi1-2*m.pi/3)*c1*s1)+ w2*c2**2*(1+s2**2-2*m.sqrt(2)*sym.cos(phi2-2*m.pi/3)*c2*s2))
pr9 = (1/6)*(w1*(c1**4+s1**4-2*sym.cos(2*phi1+2*m.pi/3)*c1**2*s1**2)+ w2*(c2**4+s2**4-2*sym.cos(2*phi2+2*m.pi/3)*c2**2*s2**2))
pr = [pr1, pr2,pr3, pr4,pr5, pr6,pr7, pr8,pr9]      # list of analytic prob.

w = m.e**((2/3)*m.pi*(1j))     # third root of unity
POVM_vec = (1/(2**.5))*(np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]))  # an array of POVM direction vectors
POVM_elts = [(1/3)*np.outer(np.conjugate(POVM_vec[i]),POVM_vec[i]) for i in range(len(POVM_vec))]   # a list of POVM matrices


In [4]:
#rough
rho = fn.Creating_states(Abstract=1)[2]             # abstract density matrix
sym_probs = [sym.trace(rho*i) for i in POVM_elts]   #Theoretical Abstract Expressions for Probabilities (Computer Generated)



In [5]:
# GD main function, using prof's expressions
def GD_search_lib(N = 10000, params = [0,m.pi/2,0,0,m.pi/4] , guess_params = [.1, .3, 0, 0, m.pi/4], show_calcs = False):
    bounds = [(-m.pi,m.pi+.1), (-m.pi,m.pi+.1) ,(-m.pi,m.pi+.1),(-m.pi,m.pi+.1),(0,m.pi/2+.1)]
    num_expt = fn.num_experiment(N = N, params = params)
    states, rho, th_pr = num_expt[0], num_expt[1], num_expt[3] # for later
    nj_vec = num_expt[4]
    global pr                                           # importing the Analytical probabilities
    lnL_list = [nj_vec[i]*sym.log(pr[i]) for i in range(len(pr))]     
    lnL_abst = sum(lnL_list).evalf()                                  
    def lnL(sp = guess_params):                                    #s_p stands for search parameters
        lnL = lnL_abst.subs({theta1: sp[0], theta2: sp[1] , phi1: sp[2] , phi2: sp[3] , alpha: sp[4] })
        neg_likelihood = -(lnL.evalf())
        return neg_likelihood 
    #grad function
    def grad(p=[0,m.pi/2,0,0,m.pi/4] , s=(1/500), show_calcs = False):    # "point" is the list of parameters
        del_theta1 = lnL(sp = [p[0]+s,p[1],p[2],p[3],p[4]]) - lnL(sp = [p[0],p[1],p[2],p[3],p[4]])
        del_theta2 = lnL(sp = [p[0],p[1]+s,p[2],p[3],p[4]]) - lnL(sp = [p[0],p[1],p[2],p[3],p[4]])
        del_phi1   = lnL(sp = [p[0],p[1],p[2]+s,p[3],p[4]]) - lnL(sp = [p[0],p[1],p[2],p[3],p[4]])
        del_phi2   = lnL(sp = [p[0],p[1],p[2],p[3]+s,p[4]]) - lnL(sp = [p[0],p[1],p[2],p[3],p[4]])
        del_alpha  = lnL(sp = [p[0],p[1],p[2],p[3],p[4]+s]) - lnL(sp = [p[0],p[1],p[2],p[3],p[4]])

        grad = [del_theta1/s,del_theta2/s,del_phi1/s,del_phi2/s,del_alpha/s]
        grad = [i if abs(i) > 1e-5 else 0 for i in grad]
        return grad
    def GD(p0 = [0,m.pi/2,0,0,m.pi/4], nit = 20, step = 1/120, show_calc = False):
        cost0 = lnL(p0)
        s = step
        grad_new = grad(p = p0)
        params_new = p0
        values = []
        for i in range(nit):
            grad_mod = (sum([abs(i)**2 for i in grad_new]))**(.5)
            params_new = [params_new[i]-s*(grad_new[i]/grad_mod) for i in range(len(p0))]
            grad_new = grad(params_new)
            value = [lnL(params_new), params_new, grad_new, i]
            values.append(value)
            if i > (3/10)*nit//1 and i > 10:
                s = 5*step
                if i > (8/10)*nit//1:
                    s = 15*step
            if show_calc == True:
                print(value,'\n ')
        return params_new

    # GDresult = GDlib(lnL, params0, bounds = bounds)
    opt = GDlib(lnL, guess_params, bounds = bounds)
    opt_params = opt.x
    
    # now to find the Fidelity
    opt_states_raw = fn.Creating_states(opt_params)[0]              #new states
    
    fid0_sym, fid1_sym = (abs(opt_states_raw[0].H*states[0])**2).evalf(), (abs(opt_states_raw[1].H*states[1])**2).evalf()
    fid0, fid1 = complex(fid0_sym[0]).real, complex(fid1_sym[0]).real
    fid0_cross_sym, fid1_cross_sym = (abs(opt_states_raw[1].H*states[0])**2).evalf(), (abs(opt_states_raw[0].H*states[1])**2).evalf()
    fid0_cross, fid1_cross = complex(fid0_cross_sym[0]).real, complex(fid1_cross_sym[0]).real

    flip_algo = 0
    if fid0_cross + fid1_cross > fid0+fid1:                     # adding zero because of type being matrix..
        opt_params_1 = [opt_params[1],opt_params[0],opt_params[2],opt_params[3],opt_params[4]]        # reversing oder of theta1 and theta2 if cross fidelities are higher.
        flip_algo= 1
        # delta = abs(opt_params_1[0]-opt_params_1[1])/4                                                  # creating a buffer for new bounds
        # bound2 = [(i-delta, i+delta) for i in opt_params_1]                                             # buffer of size delta around each parameter
        # opt_2 = GDlib(lnL, opt_params_1, bounds = bound2)
        # opt_params_2 = opt_2.x
        # opt_params= opt_params_2
        opt_params_2 = GD(opt_params_1)
        opt_params= opt_params_2

    opt_states = fn.Creating_states(opt_params)[0]         #new correctly switched states
    fid0_sym, fid1_sym = (abs(opt_states[0].H*states[0])**2).evalf(), (abs(opt_states[1].H*states[1])**2).evalf()
    fid0, fid1 = complex(fid0_sym[0]).real, complex(fid1_sym[0]).real
    fid = [fid0, fid1]

    if show_calcs == True:
        # pprint(rho)
        # print("collapse vector",nj_vec, "\ngiven parameters\n", params, "\noptimal parameters\n", opt_params)
        # print('\noriginal states: \n' )
        # pprint(states)
        print('\n raw states:' )
        print(list(opt_states_raw[0]))
        print(list(opt_states_raw[1]))
        print('\n final states:')
        print(list(opt_states[0]))
        print(list(opt_states[1]))
        print("\n raw params ", opt.x)
        if flip_algo == 1:
            print("\n intermediate params ", opt_params_1)
        print("\n final params ", opt_params)
    
        if flip_algo == 1:
            print("\n Fidelities raw", [fid0_cross, fid1_cross])
        print("\n Fidelities final",fid)
        
    return ['OPTIMIZATION (1st GDsearch):',opt,'\n\n',opt_params,opt_states,fid]


In [6]:
# generalizing creating states function to complex states
params00 = [0,m.pi/2,0,0,m.pi/4]
params00complex = [0,m.pi/2,0,m.pi/2,m.pi/4]

a = (fn.Creating_states(params00complex)[2])
pprint(a)
#fucntion verified 

⎡0.5             0             0⎤
⎢                               ⎥
⎢          6.28318530717959⋅ⅈ   ⎥
⎢ 0   0.5⋅ℯ                    0⎥
⎢                               ⎥
⎣ 0              0             0⎦


In [7]:
# Generalizing num expt funciton to Complex states

w = m.e**((2/3)*m.pi*(1j))     # third root of unity
POVM_vec = (1/(2**.5))*(np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]))  # an array of POVM direction vectors
POVM_elts = [(1/3)*np.outer(np.conjugate(POVM_vec[i]),POVM_vec[i]) for i in range(len(POVM_vec))]   # a list of POVM matrices
M = [[np.trace(np.dot(POVM_elts[i],POVM_elts[j])) for i in range(len(POVM_elts))] for j in range(len(POVM_elts))]     # creating M matrix using POVM definition
u_0 = [1/3 for i in range(9)]           # cerating u_0 vector, to create the inverse matrix
M_inv = 3*np.outer(u_0,u_0) + 12*(np.eye(9) - np.outer(u_0,u_0))        # creating the inverse matrix

def num_experiment(N = 10000, params = [0,m.pi/2,0,0,m.pi/4], show_calcs = False, seed = None):
    if seed is not None:
        rand.seed(seed)
    
    creation = fn.Creating_states( params = params, Abstract = 0) # theoretical rho #@@@
    states = creation[0]
    rho = creation[-1]
    prob_vec_sympy =  [np.trace(np.dot(POVM_elts[i],rho)) for i in range(9)]    # created list of Th. probabilities
    prob_vec_raw = [(float(i.as_real_imag()[0])+float(i.as_real_imag()[1])*1j) for i in prob_vec_sympy]  # this is to avoid error, to convert sympy float to ordinary number
    prob_vec = [round(i.real, 10) for i in prob_vec_raw if abs(i.imag) < .0001]          # cleaned up theoretical prob vector

    POVM_dir_symbols = ['d1','d2','d3','d4','d5','d6','d7','d8','d9']      # symbols to indicate collapsed direction
    #prob distribution is simply the corresponding elements of the prob_vec
    collapse_dir_vec = rand.choices(POVM_dir_symbols, weights=prob_vec, k = N)   # choosing collapse directions with weights for N trials
    nj_vec = [collapse_dir_vec.count(f'd{i+1}') for i in range(9)]
    pj_num_vec = [i/N for i in nj_vec]                                  # numerical prob vector     

    r_vec = np.dot(M_inv,pj_num_vec)
    rho_num_list = [r_vec[i]*POVM_elts[i] for i in range(len(POVM_elts))]   # list of matrices, see equation 7 in notes pair_disc.pdf
    rho_num = np.zeros_like(rho_num_list[0])
    # Loop for reconstructing the numerical matrix
    for i in rho_num_list:
        rho_num = np.add(rho_num, i)

    if show_calcs == True: 
        print('rho:')
        pprint(rho)
        print("\n rho_num \n", rho_num)
        print("\n prob vec", prob_vec)
        print("\n nj vec",nj_vec)
        print("\n sum pj, sum nj ", sum(prob_vec), sum(nj_vec))
    
    return [states, rho, rho_num, prob_vec, nj_vec]

# res = num_experiment(N = 10000, params= params00complex, show_calcs = 1)
# res[0]
# res[1]
# res[2]
# res[3]
# res[4]
# fucntion verified

In [8]:
# generalizing inversion function to complex states
# Fidelity Function
def Inversion( N = 10000, params = [0,m.pi/2, m.pi/4], threshold = 'variable'):    # coeffs are as list [theta_1, theta_2, alpha] in radians
    experiment = fn.num_experiment(N= N, params = params, show_calcs= False)
    rho = experiment[1]
    rho_num = experiment[2]                                                # !!! can enhance using postprocessing here (2*rho_num = rho_num+ conj(rho_num)). can also do directly in num_expt funciton
    eigenvalues, eigenvectors = np.linalg.eig(rho_num)
    evals = np.array([i.real for i in eigenvalues if abs(i.imag)< .0001])  # !!! does this rounding affect things later..?
    evecs = np.around(eigenvectors, decimals=9)
    min_eval = np.min([abs(i) for i in evals])
    def solve_quadratic(a, b, c):
        d = (b**2) - (4*a*c)
        sol1 = (-b-np.sqrt(d))/(2*a)
        sol2 = (-b+np.sqrt(d))/(2*a)
        return [sol1, sol2]
    if len(params) ==3:                                 # For studying the simpler case with phis = 0
        a0, a1 = np.cos(params[0]), np.sin(params[0])
        b0, b1 = np.cos(params[1]), np.sin(params[1])
    elif len(params) == 5:
        a0, a1 = np.cos(params[0]), np.exp(params[2]*1j)*np.sin(params[0])
        b0, b1 = np.cos(params[1]), np.exp(params[3]*1j)*np.sin(params[1])
    c = [a0, a1, b0, b1]                                # four list of state coefficients

    if threshold != 'variable':
        if type(threshold) is str:
            print("Error! Threshold should be a number ")
    else:
        threshold = (4.2)/(N**.5)           # variable threshold


    
    xi = evecs[:, np.argmin([abs(i) for i in evals])]     # xi vector is same as the perp vector 
    c00,c01,c11 = xi[0],xi[2],xi[1]                         # defining In this weird way coz new basis arrangement, last elt is the |+> basis therefore c01 = xi[2]
    # the reconstructing the states' coefficients if-else loops
    c_raw = 0
    if abs(c00) > threshold and abs(c11) > threshold: 
        # print("condition 1")              
        if abs(c11)/abs(c00) < 2:
            soln  =  solve_quadratic(c00, np.sqrt(2)*c01, c11)
            # soln  =  solve_quadratic(c00, 2*c01, c11)
            z_a,z_b = soln[0], soln[1]
            a1 = (1/(np.sqrt(1+abs(z_a)**2)))           
            a0 = np.conj(z_a)*a1
            b1 = (1/(np.sqrt(1+abs(z_b)**2)))
            b0 = np.conj(z_b)*b1
            c_raw = [a0,a1,b0,b1]

        else:
            soln  =  solve_quadratic(c11, np.sqrt(2)*c01, c00)
            # soln  =  solve_quadratic(c11, 2*c01, c00)
            z_a,z_b = soln[0], soln[1]
            a0 = (1/(np.sqrt(1+abs(z_a)**2)))                          
            a1 = np.conj(z_a)*a0
            b0 = (1/(np.sqrt(1+abs(z_b)**2)))
            b1 = np.conj(z_b)*b0
            c_raw = [a0,a1,b0,b1]
            # print(z_a,z_b)
            # print('coeffs',a0,a1,b0,b1)
    elif abs(c00) <= threshold and abs(c11) > threshold:      # some cases are excluded if we don't use eqality
        a0, a1 = 1,0        # random choice, matching later
        # z = np.conj((-ca settrecer11)/(2*c01))     # z defined as (a0*/a1*)
        z = np.conj((-c11)/(np.sqrt(2)*c01))     # z defined as (a0*/a1*)
        b1= (1/(np.sqrt(1+abs(z)**2))) 
        b0= np.conj(z)*b1
        # print('condition #2')
        c_raw = [a0,a1,b0,b1]
    elif abs(c00) > threshold and abs(c11) <= threshold:
        a0, a1 = 0,1        
        # z = np.conj((-2*c01)/(c00))      # z defined as (b0*/b1*)
        z = np.conj((-(np.sqrt(2))*c01)/(c00))      # z defined as (b0*/b1*)
        b1= (1/(np.sqrt(1+abs(z)**2))) 
        b0= np.conj(z)*b1
        # print('condition #3, z = ',z)
        c_raw = [a0,a1,b0,b1]
    elif abs(c00) <= threshold and abs(c11) <= threshold:
        a0, a1 = 0,1        
        b0, b1 = 1,0     
        c_raw = [a0,a1,b0,b1]

    f00,f01 = (abs((c_raw[0]*c[0]+c_raw[1]*c[1])))**2, (abs((c_raw[0]*c[2]+c_raw[1]*c[3])))**2
    f10,f11 = (abs((c_raw[2]*c[0]+c_raw[3]*c[1])))**2, (abs((c_raw[2]*c[2]+c_raw[3]*c[3])))**2

    if  (f00 + f11) > (f01+ f10) :       
        c_num = c_raw
    else:
        c_num = [c_raw[2],c_raw[3],c_raw[0],c_raw[1]]

    F_a = (abs(sum([np.conj(c[i])*c_num[i] for i in range(0,2)])))**2
    F_b = (abs(sum([np.conj(c[i])*c_num[i] for i in range(2,4)])))**2
    Inversion_Fid = [F_a, F_b]


    # finding final parameters from the final coeffs
    tht1 = np.arccos(abs(c_num[0]))      # magnutude coz by construction any phase is pushed into the global phase (which also changes the relative phase)
    tht2 = np.arccos(abs(c_num[2]))

    #finding the phi angles
    phi_angles = np.angle(c_num)
    # angles = (180/m.pi)*angles  #in degrees
    phi_corrections = [180*(m.pi/180) if 90*(m.pi/180) < i < 180*(m.pi/180) or 270*(m.pi/180) < i < 360*(m.pi/180) else 0 for i in [tht1, tht2]]
    del_phis = [(phi_angles[0]-phi_angles[1])-phi_corrections[0], (phi_angles[2]-phi_angles[3])-phi_corrections[1]]
    phi1, phi2 = del_phis[0], del_phis[1]
    
    #finding the priors, see notebook
    psi0_num, psi1_num = [c_num[0], c_num[1]], [c_num[2], c_num[3]]
    psi0psi0_num = np.array([psi0_num[0]*psi0_num[0], psi0_num[1]*psi0_num[1], (np.sqrt(2)*psi0_num[0]*psi0_num[1])], dtype = complex) 
    psi1psi1_num = np.array([psi1_num[0]*psi1_num[0], psi1_num[1]*psi1_num[1], (np.sqrt(2)*psi1_num[0]*psi1_num[1])], dtype = complex)              # @@@ the square rooting is simplistic. see notebook for better. (cross term last due to prof's basis arrangement)
    overlap_num = np.dot(np.conjugate(psi0psi0_num),psi1psi1_num)
    psi1_cross_psi1_num = np.outer(psi1psi1_num, np.conjugate(psi1psi1_num))
    prior0 = (np.dot(np.dot(np.conjugate(psi0psi0_num), (rho_num - psi1_cross_psi1_num)), psi0psi0_num))/(1-abs(overlap_num)**2)        # see derivation in the notebook
    alpha = np.arccos(np.sqrt(prior0))
  
    psi0_cross_psi0_num = np.outer(psi0psi0_num, np.conjugate(psi0psi0_num))        # trying using the arcsin() of root(p1), earlier it was using the arc cos of root p0
    prior1 = (np.dot(np.dot(np.conjugate(psi1psi1_num), (rho_num - psi0_cross_psi0_num)), psi1psi1_num))/(1-abs(overlap_num)**2)        # see derivation in the notebook
    alpha_1 = np.arcsin(np.sqrt(prior1))
    
    final_params = [tht1, tht2, phi1, phi2, alpha]  

    return [Inversion_Fid, params ,final_params]


In [9]:
#RoughPad: testing the quality of output parameters for the inversion code
# Inversion(params = [m.pi/4,m.pi/2+m.pi/4, m.pi/4])      # eg.
# Inversion(params = [0,m.pi/2, m.pi/4])
# Inversion(params = [0,m.pi/4, m.pi/4])
# Inversion(params = [0,m.pi/8, m.pi/4])
# Inversion(params = [m.pi/2,m.pi/2+m.pi/4, m.pi/4])
# Inversion(params = [m.pi/2+m.pi/4,m.pi-.1, m.pi/4])
Inversion(params = [m.pi,m.pi+m.pi/6, m.pi/4])

TypeError: num_experiment() got an unexpected keyword argument 'show_calcs'

In [ ]:
# Trying combination Inversion+ Grad Descent
input_params = [ m.pi/4, m.pi/2-m.pi/8 , 0,0, m.pi/4]
inv = Inversion(params = input_params)
init = inv[2]
print('','inversion: ',inv)


inv_fid = inv[0]
print('','\n',)
hybrid_fid = GD_search_lib(params = input_params, guess_params = init)[-1]
hybrid_params = [(180/m.pi)*i for i in GD_search_lib(params = input_params, guess_params = init)[1].x]

print('','\n input params', [(180/m.pi)*i for i in input_params])
print('','inversion params', [(180/m.pi)*i for i in init])
print('','hybrid params',hybrid_params)
print('','\n hybrid fid',hybrid_fid)

 inversion:  [[0.9758498186399057, 0.9916868574031389], [0.7853981633974483, 1.1780972450961724, 0, 0, 0.7853981633974483], [0.8380306998446552, 1.1167772111824368, 0.29432953652509225, 0.1810960724652635, (1.2626899041355861-5.60466663954038e-17j)]]
 



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_differentiable_functions.py:107: ComplexWarning: Casting complex values to real discards the imaginary part
  self.x = np.atleast_1d(x0).astype(float)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_lbfgsb_py.py:335: ComplexWarning: Casting complex values to real discards the imaginary part
  x = array(x0, float64)


KeyboardInterrupt: 

In [ ]:
def Hybrid(input_params):             #Combination of the inversing and gradient descent.
    inv = Inversion(params = input_params)
    init = inv[2]               # guess from inversion method
    inv_fid = inv[0]            #inversion fid

    hybrid_fid = GD_search_lib(params = input_params, guess_params = init)[-1]
    hybrid_params = [(180/m.pi)*i for i in GD_search_lib(params = input_params, guess_params = init)[1].x]

    print('','\n input params', [(180/m.pi)*i for i in input_params])
    
    print('','inversion params', [(180/m.pi)*i for i in init])
    print('','hybrid params',hybrid_params)
    
    print('','\n inversion fid', inv_fid)
    print('','\n hybrid fid',hybrid_fid)


In [ ]:
# Hybrid([0,m.pi/2, 0,0, m.pi/4])
# Hybrid([ m.pi/4, m.pi/2-m.pi/8 , 0,0, m.pi/4])

In [ ]:
#Checking whether the error due to the likelihood function being lower than the theoretical minimum is because of some error
# in the professor's calculations of abstract probability. So doing it alternatively using computer generated theoretical versions
# of the probabilities and seeing if the error lasts.
# correct values  =  [m.pi/4, m.pi/2-m.pi/8 , 0,0, m.pi/4]  # for this particular case
nj = [142, 128, 426, 2117, 1235, 1326, 2052, 1210, 1364]
nj_norm = [i/sum(nj) for i in nj]
optimized_params = [ 0.5425531 ,  1.07939737, -1.0250692 , -0.00644445,  0.97872786]
opt_fun = 19292.04  #same as 1.929925
theoretical_min_fun = -sum([i*m.log(i) for i in nj_norm])    #optimized function shouldnt be lesser than this value

# first: using prof equations
lnL_list_prof = [nj_norm[i]*sym.log(pr[i]) for i in range(9)]      #list of lnl for each part
lnl_list_prof_truval = [i.subs({theta1: m.pi/4 , theta2: m.pi/2-m.pi/8, phi1:0, phi2:0, alpha: m.pi/4}) for i in lnL_list_prof]      #subbing in the true parameters
lnl_list_prof_opt_params  = [i.subs({theta1:0.5425531 , theta2: 1.07939737, phi1:-1.0250692, phi2:-0.00644445, alpha: 0.97872786}) for i in lnL_list_prof]      #subbing in the "optimal" params

lnL_prof_truval = sum(lnl_list_prof_truval)
lnL_prof_opt_params = sum(lnl_list_prof_opt_params)

lnL_list_comp = [nj_norm[i]*sym.log(sym_probs[i]) for i in range(9)]      #list of lnl for each part, This time using the sympy-generated expressions.
lnl_list_comp_truval = [i.subs({theta1: m.pi/4 , theta2: m.pi/2-m.pi/8, phi1:0, phi2:0, alpha: m.pi/4}) for i in lnL_list_comp]      #subbing in the true parameters
lnl_list_comp_opt_params  = [i.subs({theta1:0.5425531 , theta2: 1.07939737, phi1:-1.0250692, phi2:-0.00644445, alpha: 0.97872786}) for i in lnL_list_comp]      #subbing in the "optimal" params

lnL_comp_truval = sum(lnl_list_comp_truval)
lnL_comp_opt_params = sum(lnl_list_comp_opt_params).evalf()

print('','Theoretical min likelihood', theoretical_min_fun)          # can't exceed this value

print('','lnL at the input params, using prof expressions', lnL_prof_truval)
print('','lnL at opt params, using prof expressions', lnL_prof_opt_params)

print('','lnL at the input params, using sympy expressions', lnL_prof_truval)
print('','lnL at opt params, using sympy expressions', lnL_prof_opt_params)

 Theoretical min likelihood 1.9578020323711336
 lnL at the input params, using prof expressions -1.97026185324546
 lnL at opt params, using prof expressions -1.92920467947100
 lnL at the input params, using sympy expressions -1.97026185324546
 lnL at opt params, using sympy expressions -1.92920467947100


In [ ]:
"""def lnL(p):                                    #s_p stands for search parameters
        global pr                                           # importing the Analytical probabilities
        
        
        lnL_list = [nj_vec[i]*sym.log(pr[i]) for i in range(len(pr))]
        lnL_abst = sum(lnL_list).evalf()                                  
        lnL = lnL_abst.subs({theta1: sp[0], theta2: sp[1] , phi1: sp[2] , phi2: sp[3] , alpha: sp[4] })
        neg_likelihood = -(lnL.evalf())
        return neg_likelihood """

In [ ]:
# GD main function,using the SymPy method
def GD_search_lib_comp(N = 10000, params = [0,m.pi/2,0,0,m.pi/4] , guess_params = [.1, .3, 0, 0, m.pi/4], show_calcs = False):
    bounds = [(-m.pi,m.pi+.1), (-m.pi,m.pi+.1) ,(-m.pi,m.pi+.1),(-m.pi,m.pi+.1),(0,m.pi/2+.1)]
    num_expt = fn.num_experiment(N = N, params = params)
    states, rho, th_pr = num_expt[0], num_expt[1], num_expt[3] # for later
    nj_vec = num_expt[4]
    lnL_list = [nj_vec[i]*sym.log(sym_probs[i]) for i in range(9)]       #CHANGING TO SYMPY EXPRESSIONS
    lnL_abst = sum(lnL_list).evalf()                                  
    def lnL(sp = guess_params):                                    
        lnL = lnL_abst.subs({theta1: sp[0], theta2: sp[1] , phi1: sp[2] , phi2: sp[3] , alpha: sp[4] })
        neg_likelihood = -(lnL.evalf())
        return neg_likelihood 
    #grad function
    def grad(p=[0,m.pi/2,0,0,m.pi/4] , s=(1/500), show_calcs = False):    # "point" is the list of parameters
        del_theta1 = lnL(sp = [p[0]+s,p[1],p[2],p[3],p[4]]) - lnL(sp = [p[0],p[1],p[2],p[3],p[4]])
        del_theta2 = lnL(sp = [p[0],p[1]+s,p[2],p[3],p[4]]) - lnL(sp = [p[0],p[1],p[2],p[3],p[4]])
        del_phi1   = lnL(sp = [p[0],p[1],p[2]+s,p[3],p[4]]) - lnL(sp = [p[0],p[1],p[2],p[3],p[4]])
        del_phi2   = lnL(sp = [p[0],p[1],p[2],p[3]+s,p[4]]) - lnL(sp = [p[0],p[1],p[2],p[3],p[4]])
        del_alpha  = lnL(sp = [p[0],p[1],p[2],p[3],p[4]+s]) - lnL(sp = [p[0],p[1],p[2],p[3],p[4]])

        grad = [del_theta1/s,del_theta2/s,del_phi1/s,del_phi2/s,del_alpha/s]
        grad = [i if abs(i) > 1e-5 else 0 for i in grad]
        return grad
    def GD(p0 = [0,m.pi/2,0,0,m.pi/4], nit = 20, step = 1/120, show_calc = False):
        cost0 = lnL(p0)
        s = step
        grad_new = grad(p = p0)
        params_new = p0
        values = []
        for i in range(nit):
            grad_mod = (sum([abs(i)**2 for i in grad_new]))**(.5)
            params_new = [params_new[i]-s*(grad_new[i]/grad_mod) for i in range(len(p0))]
            grad_new = grad(params_new)
            value = [lnL(params_new), params_new, grad_new, i]
            values.append(value)
            if i > (3/10)*nit//1 and i > 10:
                s = 5*step
                if i > (8/10)*nit//1:
                    s = 15*step
            if show_calc == True:
                print(value,'\n ')
        return params_new

    # GDresult = GDlib(lnL, params0, bounds = bounds)
    opt = GDlib(lnL, guess_params, bounds = bounds)
    opt_params = opt.x
    
    # now to find the Fidelity
    opt_states_raw = fn.Creating_states(opt_params)[0]              #new states
    
    fid0_sym, fid1_sym = (abs(opt_states_raw[0].H*states[0])**2).evalf(), (abs(opt_states_raw[1].H*states[1])**2).evalf()
    fid0, fid1 = complex(fid0_sym[0]).real, complex(fid1_sym[0]).real
    fid0_cross_sym, fid1_cross_sym = (abs(opt_states_raw[1].H*states[0])**2).evalf(), (abs(opt_states_raw[0].H*states[1])**2).evalf()
    fid0_cross, fid1_cross = complex(fid0_cross_sym[0]).real, complex(fid1_cross_sym[0]).real

    flip_algo = 0
    if fid0_cross + fid1_cross > fid0+fid1:                     # adding zero because of type being matrix..
        opt_params_1 = [opt_params[1],opt_params[0],opt_params[2],opt_params[3],opt_params[4]]        # reversing oder of theta1 and theta2 if cross fidelities are higher.
        flip_algo= 1
        opt_params_2 = GD(opt_params_1)
        opt_params= opt_params_2

    opt_states = fn.Creating_states(opt_params)[0]         #new correctly switched states
    fid0_sym, fid1_sym = (abs(opt_states[0].H*states[0])**2).evalf(), (abs(opt_states[1].H*states[1])**2).evalf()
    fid0, fid1 = complex(fid0_sym[0]).real, complex(fid1_sym[0]).real
    fid = [fid0, fid1]
        
    return ['OPTIMIZATION (1st GDsearch):',opt,'\n\n',opt_params,opt_states,fid]


In [ ]:
def Hybrid(input_params):             #Combination of the inversing and gradient descent.
    inv = Inversion(params = input_params)
    init = inv[2]               # guess from inversion method
    inv_fid = inv[0]            #inversion fid

    hybrid_fid = GD_search_lib_comp(params = input_params, guess_params = init)[-1]
    hybrid_params = [(180/m.pi)*i for i in GD_search_lib(params = input_params, guess_params = init)[1].x]

    print('','\n input params', [(180/m.pi)*i for i in input_params])
    
    print('','inversion params', [(180/m.pi)*i for i in init])
    print('','hybrid params',hybrid_params)
    
    print('','\n inversion fid', inv_fid)
    print('','\n hybrid fid',hybrid_fid)

# Hybrid([0,m.pi/2, 0,0, m.pi/4])
Hybrid([ m.pi/4, m.pi/2-m.pi/8 , 0,0, m.pi/4])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_differentiable_functions.py:107: ComplexWarning: Casting complex values to real discards the imaginary part
  self.x = np.atleast_1d(x0).astype(float)


TypeError: Invalid comparison of non-real 19814.1354223994 + 4381.74871879292*I

analytical probabilities
 [0.128664005485443, 0.0495599436968722, 0.160153979689046, 0.0850873929732140, 0.0762722481755657, 0.164423419442150, 0.0898595822047441, 0.0762722481755657, 0.165555225119165]
sum analytical probabilities
 0.995848044961765
